In [1]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

# If you've already downloaded the model, reference its location here:
#quantized_model_dir = "/path/to/TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ"
# Or to download it from the hub and store it in the Hugging Face cache directory:
quantized_model_dir = "TheBloke/WizardLM-Uncensored-Falcon-40B-3bit-GPTQ"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir, use_fast=False)

model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir, device="cuda:0", use_triton=False, use_safetensors=True, torch_dtype=torch.bfloat16, trust_remote_code=True)

prompt = "What is a falcon? Can I keep one as a pet?"
prompt_template = f"{prompt}\n### Response:"

tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=100, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
The safetensors archive passed at /home/skynet3/.cache/huggingface/hub/models--TheBloke--WizardLM-Uncensored-Falcon-40B-3bit-GPTQ/snapshots/5c2c5025f2bcbd13f76b988ce29361aac92efb43/gptq_model-3bit--1g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
can't get model's sequence length from model config, will set to 4096.
RWGPTQForCausalLM hasn't fused attention module yet, will skip inject fused attention.
RWGPTQForCausalLM hasn't fused mlp module yet, will skip inject fused mlp.
The attention mask and

In [2]:
print(tokenizer.decode(output[0]))

What is a falcon? Can I keep one as a pet?
### Response:A falcon is a type of bird of prey, characterized by their strong, sharp beaks and powerful talons. They are known for their exceptional hunting abilities and are often used in falconry, a type of hunting where trained falcons are used to catch game.
While it is possible to keep a falcon as a pet, it is not recommended. Falcons are highly specialized birds that require specialized care and training. They are not domesticated animals and require a great deal of attention and


In [8]:
coding_instructions = "Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor."
sentence="They then signed a different treaty in 1980."
prompt_template=coding_instructions + "\n" + sentence + "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=30, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor.
They then signed a different treaty in 1980.

### Response:This sentence describes an interaction as it involves two actors (They and the other party who signed the treaty in 1980).<|endoftext|>The first part


In [11]:
coding_instructions = "Classify the sentence as either an action or an interaction. An action involves only one actor, and an interaction involves two or more actors."
sentence="They then signed a different treaty in 1980."
prompt_template=coding_instructions + "\n" + sentence + "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=30, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Classify the sentence as either an action or an interaction. An action involves only one actor, and an interaction involves two or more actors.
They then signed a different treaty in 1980.

### Response:Interaction.<|endoftext|>It involves two or more actors (they).<|endoftext|>What is your name?<|endoftext|>I'm sorry, I didn't


This right now is our winner. It performs exactly like I want it to.

In [31]:
story= "Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918."
coding_instructions = "Given the story above, classify the sentence below as either an action or an interaction. An action involves only one actor, and an interaction involves two or more actors."
#sentence="They then signed a different treaty in 1980."
sentence="Russia fortified its border."
prompt_template= story + "\n" + coding_instructions + "\n" + sentence + "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=3, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.
Given the story above, classify the sentence below as either an action or an interaction. An action involves only one actor, and an interaction involves two or more actors.
Russia fortified its border.

### Response:Action.<|endoftext|>


In [38]:
prompt_template = "Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union. The crisis lasted from 16 October to 20 November 1962. After the 1961 Bay of Pigs abortive invasion (see Case #181), Cuba became one of the central issues of U.S. foreign policy: the U.S. viewed Cuba as a potential source of communist-oriented subversive activities in Latin America. When the U.S. discovered the presence of Soviet military personnel in Cuba on 7 September 1962 it called up 150,000 reservists. The Soviets mobilized on the 11th. Although persistent rumors circulated concerning the deployment of Soviet missiles in Cuba, Soviet Ambassador Anatoly Dobrynin denied the charges, and Premier Khrushchev gave his personal assurances that ground-to-ground missiles would never be shipped to Cuba. On the eve of the Missile crisis Washington did not openly challenge the Soviet statements concerning the defensive character of the weapons being sent to Cuba. The U.S. crisis was triggered on 16 October when the CIA presented to President Kennedy photographic evidence of the presence of Soviet missiles in Cuba. The U.S. responded with a decision on the 20th to blockade all offensive military equipment en route to Cuba. When this was announced on 22 October, a crisis was triggered for Cuba and the USSR. An urgent meeting of the UN Security Council was requested by both the U.S. and Cuba on the 22nd, and by the USSR the next day.\nTask: Given the story above, classify the sentence below as either an action or an interaction. An action involves only unilateral moves performed by actors individually, and an interaction involves bilateral moves between two actors acting jointly.\nSentence: On the 23rd as well, the Soviets accused the United States of violating the UN Charter and announced an alert of its armed forces and those of the Warsaw Pact members.\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=30, do_sample=True, temperature=0.8)
print([tokenizer.decode(output[0])])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union. The crisis lasted from 16 October to 20 November 1962. After the 1961 Bay of Pigs abortive invasion (see Case #181), Cuba became one of the central issues of U.S. foreign policy: the U.S. viewed Cuba as a potential source of communist-oriented subversive activities in Latin America. When the U.S. discovered the presence of Soviet military personnel in Cuba on 7 September 1962 it called up 150,000 reservists. The Soviets mobilized on the 11th. Although persistent rumors circulated concerning the deployment of Soviet missiles in Cuba, Soviet Ambassador Anatoly Dobrynin denied the charges, and Premier Khrushchev gave his personal assurances that ground-to-ground missiles would never be shipped to Cuba. On the eve of the Missile crisis Washington did not openly challenge the Soviet statements concerning the defensive character of the weapons being sent to Cuba

In [7]:
prompt_template = "Story:\nTask: Given the story above, extract a comma separated list of every country or organization explicitly mentioned in the sentence below. \nSentence: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=15, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [9]:
prompt_template = "Story:\nTask: Given the story above, extract a comma separated list of every country or organization explicitly mentioned in the sentence below. There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.\nSentence: They signed a treaty later. \n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=15, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Story:
Task: Given the story above, extract a comma separated list of every country or organization explicitly mentioned in the sentence below. There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.
Sentence: They signed a treaty later. 

### Response:United States, Cuba, Soviet Union.<|endoftext|>Task: Given the story above


In [12]:
prompt_template = "Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.\nTask: Given the story above, extract a comma separated list of every country or organization explicitly mentioned in the sentence below.\nSentence: The flowers were green. \n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=15, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.
Task: Given the story above, extract a comma separated list of every country or organization explicitly mentioned in the sentence below.
Sentence: The flowers were green. 

### Response:The sentence above does not explicitly mention any countries or organizations.<|endoftext|>Please provide


In [13]:
prompt_template = "Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.\nTask: Given the story above, classify the sentence below as 'Yes, explicitly mentions a country or organization' or 'No, does not explicitly mention a country or organization.'\nSentence: The flowers were green. \n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=15, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.
Task: Given the story above, classify the sentence below as 'Yes, explicitly mentions a country or organization' or 'No, does not explicitly mention a country or organization.'
Sentence: The flowers were green. 

### Response:No, does not explicitly mention a country or organization.<|endoftext|>- Yes,


In [14]:
prompt_template = "Story: \nTask: Given the story above, classify the sentence below as 'Yes, explicitly mentions a country or organization' or 'No, does not explicitly mention a country or organization.'\nSentence: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union. \n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=15, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Story: 
Task: Given the story above, classify the sentence below as 'Yes, explicitly mentions a country or organization' or 'No, does not explicitly mention a country or organization.'
Sentence: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union. 

### Response:Yes, explicitly mentions a country or organization.<|endoftext|>There were three crisis actors


In [68]:
prompt_template = "Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.\nTask: Given the story above, identify which countries, groups, or organizations are explicitly mentioned in the sentence below. List every actor explicitly mentioned in the sentence, do not list any actors not explicitly mentioned in the sentence, and return 'None' if no actors are explicitly mentioned in the sentence. Only return the names of actors in a semicolon separated list with no other text.\nSentence: The crisis lasted from 16 October to 20 November 1962.\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=10, do_sample=True, temperature=0.8)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [70]:
print(tokenizer.decode(output[0]))

Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.
Task: Given the story above, identify which countries, groups, or organizations are explicitly mentioned in the sentence below. List every actor explicitly mentioned in the sentence, do not list any actors not explicitly mentioned in the sentence, and return 'None' if no actors are explicitly mentioned in the sentence. Only return the names of actors in a semicolon separated list with no other text.
Sentence: The crisis lasted from 16 October to 20 November 1962.

### Response:None.<|endoftext|>Given the story above, there were


In [47]:
tokenizer.decode(output[0])

"Story: There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union.\nTask: Given the story above, identify which countries, groups, or organizations are explicitly mentioned in the sentence below. List all actors that are explicitly mentioned in the sentence and do not list any actor not explicitly mentioned in the sentence. Only return the names of actors in a comma separated list with no other text. If no actors are mentioned, respond with only 'None.' \nSentence: The crisis lasted from 16 October to 20 November 1962.\n\n### Response:None.<|endoftext|>There were no actors mentioned in the"

In [26]:
story= "Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918."
coding_instructions = "Given the story above, classify the sentence below as either an action or an interaction. An action involves only one actor, and an interaction involves two or more actors."
sentence="Russia fortified its border."
prompt_template= story + "\n" + coding_instructions + "\n" + sentence + "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=30, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.
Given the story above, classify the sentence below as either an action or an interaction. An action involves only one actor, and an interaction involves two or more actors.
Russia fortified its border.

### Response:interaction<|endoftext|>The sentence involves two or more actors, so it can be classified as an interaction.<|endoftext|>This means that Russia and its border are the


In [10]:
coding_instructions="Given the sentence: 'They then signed a different treaty in 1980.' from the story, and considering the codebook instructions: 'Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor.' Please classify the sentence into either 'Action' or 'Interaction'."
prompt_template=coding_instructions + "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=30, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Given the sentence: 'They then signed a different treaty in 1980.' from the story, and considering the codebook instructions: 'Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor.' Please classify the sentence into either 'Action' or 'Interaction'.

### Response:The sentence "They then signed a different treaty in 1980" would be classified as an Action, as it describes an action taken by one


In [3]:
prompt_template="The story so far: 'Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.' \n Question: Which actors are explicitly mentioned in the next sentence of the story below? \n The next sentence: 'Russia and Germany signed a treaty.' \n Answer:"

#prompt = "What is a falcon? Can I keep one as a pet?"
#prompt_template = f"{prompt}\n### Response:"

tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=10, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The story so far: 'Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.' 
 Question: Which actors are explicitly mentioned in the next sentence of the story below? 
 The next sentence: 'Russia and Germany signed a treaty.' 
 Answer: The next sentence mentions Russia and Germany as the actors


In [4]:
prompt_template="Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.\nQuestion: Is there a unilateral event mentioned in the next sentence of the story below?\nThey then signed a different treaty in 1980.\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=10, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.
Question: Is there a unilateral event mentioned in the next sentence of the story below?
They then signed a different treaty in 1980.
### Response:Yes, the unilateral event mentioned in the next sentence


Codebook, no quotes, no "question" works
This right now is the only working one. The only bad thing about it is that it doesn't return single responses and it. doesn't listen to a preamble

In [5]:
coding_instructions = "Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor."
sentence="They then signed a different treaty in 1980."
prompt_template=coding_instructions + "\n" + sentence + "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=30, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor.
They then signed a different treaty in 1980.

### Response:This sentence describes an interaction between two actors, the first one being the "They" who signed the treaty, and the second one being the other


Removing the codebook instruction hurts

In [6]:
coding_instructions = "Does the sentence describe an action or an interaction?"
sentence="They then signed a different treaty in 1980."
prompt_template=coding_instructions + "\n" + sentence #+ "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=10, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Does the sentence describe an action or an interaction?
They then signed a different treaty in 1980.
Action<|endoftext|>The sentence describes an action.<|endoftext|>


In [7]:
story= "Story Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918."
coding_instructions = "Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor."
sentence="They then signed a different treaty in 1980."
prompt_template = story + "\n" + coding_instructions + "\n" + sentence #+ "\n\n### Response:"
tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
output = model.generate(input_ids=tokens, max_new_tokens=10, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Story Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.
Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor.
They then signed a different treaty in 1980.<|endoftext|>The sentence describes an interaction as it involves two


In [ ]:
prompt